In [1]:
import warnings
warnings.filterwarnings("ignore")

import sys
import os
import tensorflow as tf

# sys.path.append("../libs")
sys.path.insert(1, '../')

from libs import input_data
from libs import models
from libs import trainer
from libs import freeze

W0121 21:38:46.028698 15200 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0121 21:38:46.031691 15200 deprecation_wrapper.py:119] From ..\libs\trainer.py:13: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.



In [2]:
flags=tf.app.flags

In [3]:
flags=tf.app.flags
#Important Directories
flags.DEFINE_string('data_dir','..\\..\\_inputs\\raw','Train Data Folder')
flags.DEFINE_string('summaries_dir','..\\..\\summaries','Summaries Folder')
flags.DEFINE_string('train_dir','..\\..\\logs&checkpoint','Directory to write event logs and checkpoint')
flags.DEFINE_string('models_dir','..\\..\\models','Models Folder')
#Task Specific Parameters
flags.DEFINE_string('wanted_words','yes,no,up,down,left,right,on,off,stop,go','Wanted Words')
flags.DEFINE_float('validation_percentage',10,'Validation Percentage')
flags.DEFINE_float('testing_percentage',10,'Testing Percentage')
flags.DEFINE_integer('sample_rate',16000,'Sample Rate')
flags.DEFINE_integer('clip_duration_ms',1000,'Clip Duration in ms')
flags.DEFINE_float('window_size_ms',40,'How long each spectogram timeslice is')
flags.DEFINE_float('window_stride_ms',20.0,'How far to move in time between frequency windows.')
flags.DEFINE_integer('dct_coefficient_count',40,'How many bins to use for the MFCC fingerprint')
flags.DEFINE_float('time_shift_ms',100.0,'Range to randomly shift the training audio by in time.')

FLAGS=flags.FLAGS

In [4]:
model_architecture='c_rnn'
start_checkpoint=None
logging_interval=10
eval_step_interval=1000
save_step_interval=1
silence_percentage=10.0
unknown_percentage=10.0
background_frequency=0.8
background_volume=0.3
learning_rate='0.0005,0.0001,0.00002' #Always seperated by comma, trains with each of the learning rate for the given number of iterations
train_steps='10000,10000,10000' #Declare  the training steps for which the learning rates will be used
batch_size=256
model_size_info=[48, 10, 4, 2, 2, 2, 60, 84]

In [5]:
remaining_args = FLAGS([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

In [6]:
train_dir=os.path.join(FLAGS.data_dir,'train','audio')

In [7]:
model_settings = models.prepare_model_settings(
      len(input_data.prepare_words_list(FLAGS.wanted_words.split(','))),
      FLAGS.sample_rate, FLAGS.clip_duration_ms, FLAGS.window_size_ms,
      FLAGS.window_stride_ms, FLAGS.dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
      train_dir, silence_percentage, unknown_percentage,
      FLAGS.wanted_words.split(','), FLAGS.validation_percentage,
      FLAGS.testing_percentage, model_settings,use_silence_folder=True)

..\..\_inputs\raw\train\audio\*\*.wav


W0121 21:39:28.937418 15200 deprecation_wrapper.py:119] From ..\libs\input_data.py:305: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0121 21:39:28.947395 15200 deprecation_wrapper.py:119] From ..\libs\input_data.py:306: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



..\..\_inputs\raw\train\audio\_background_noise_\doing_the_dishes.wav
..\..\_inputs\raw\train\audio\_background_noise_\dude_miaowing.wav
..\..\_inputs\raw\train\audio\_background_noise_\exercise_bike.wav
..\..\_inputs\raw\train\audio\_background_noise_\pink_noise.wav
..\..\_inputs\raw\train\audio\_background_noise_\running_tap.wav
..\..\_inputs\raw\train\audio\_background_noise_\white_noise.wav
Tensor("AudioSpectrogram:0", shape=(?, 49, 513), dtype=float32)


In [8]:
def get_train_data(args):
    sess=args
    time_shift_samples = int((FLAGS.time_shift_ms * FLAGS.sample_rate) / 1000)
    train_fingerprints, train_ground_truth = audio_processor.get_data(
        batch_size, 0, model_settings,background_frequency,
        background_volume, time_shift_samples, 'training', sess)
    return train_fingerprints,train_ground_truth

def get_val_data(args):
    '''
    Input: (sess,offset)
    '''
    sess,i=args
    validation_fingerprints, validation_ground_truth = (
            audio_processor.get_data(batch_size, i, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
    return validation_fingerprints,validation_ground_truth

In [9]:
# def get_test_data(args):
#     '''
#     Input: (sess,offset)
#     '''
#     sess,i=args
#     test_fingerprints, test_ground_truth = audio_processor.get_data(
#         batch_size, i, model_settings, 0.0, 0.0, 0, 'testing', sess)
#     return test_fingerprints,test_ground_truth

In [10]:
def main(_):
    sess=tf.InteractiveSession()
    # Placeholders
    fingerprint_size = model_settings['fingerprint_size']
    label_count = model_settings['label_count']
    fingerprint_input = tf.placeholder(
      tf.float32, [None, fingerprint_size], name='fingerprint_input')
    ground_truth_input = tf.placeholder(
      tf.float32, [None, label_count], name='groundtruth_input')
    set_size = audio_processor.set_size('validation')
    label_count = model_settings['label_count']
    
    # Create Model
    
    logits, dropout_prob = models.create_model(
      fingerprint_input,
      model_settings,
      model_architecture,
      model_size_info=model_size_info,
      is_training=True)
    
    #Start Training
    extra_args=(dropout_prob,label_count,batch_size,set_size)
    trainer.train(sess,logits,fingerprint_input,ground_truth_input,get_train_data,
                  get_val_data,train_steps,learning_rate,eval_step_interval, logging_interval=logging_interval,
                  start_checkpoint=start_checkpoint,checkpoint_interval=save_step_interval,
                  model_name=model_architecture,train_dir=FLAGS.train_dir,
                  summaries_dir=FLAGS.summaries_dir,args=extra_args)

In [11]:
tf.app.run(main=main)

W0121 21:40:23.308114 15200 deprecation_wrapper.py:119] From ..\libs\models.py:667: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0121 21:40:23.325647 15200 deprecation.py:506] From ..\libs\models.py:677: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W0121 21:40:23.337582 15200 deprecation.py:323] From ..\libs\models.py:701: GRUCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
W0121 21:40:23.339581 15200 deprecation.py:323] From ..\libs\models.py:712: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
I

KeyboardInterrupt: 

In [12]:
# save_checkpoint='../logs&checkpoint/c_rnn/ckpt-42000'

In [13]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-batched.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True,model_size_info=model_size_info)

In [14]:
# save_path=os.path.join(FLAGS.models_dir,model_architecture,'%s-batched.pb'%os.path.basename(save_checkpoint))
# freeze.freeze_graph(FLAGS,model_architecture,save_checkpoint,save_path,batched=True,model_size_info=model_size_info)